# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,274.40,100,75,1.03,CL,1717694499
1,1,iqaluit,63.7506,-68.5145,276.00,93,75,2.06,CA,1717694362
2,2,holualoa,19.6228,-155.9522,299.74,86,40,0.00,US,1717694257
3,3,tiksi,71.6872,128.8694,278.31,78,60,1.18,RU,1717694500
4,4,howard springs,-12.4970,131.0470,293.10,63,0,4.12,AU,1717694500


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    size = "Humidity",
    geo = True,
    tiles = "OSM",
    color = "City",
    alpha = 0.35,
    frame_width = 700,
    frame_height = 500,
    )

# Display the map
map_plot_1

c:\Users\rdesh\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
my_cities_df = city_data_df.loc[city_data_df["Cloudiness"] == 0,:]
my_cities_df = my_cities_df.loc[city_data_df["Max Temp"] > 250,:]
my_cities_df = my_cities_df.loc[my_cities_df["Wind Speed"] < 2]
# Drop any rows with null values
my_cities_df.dropna()
# Display sample data
my_cities_df.sample(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
435,435,ad dabbah,18.0500,30.9500,313.52,9,0,1.11,SD,1717694711
153,153,marsa alam,25.0757,34.8918,304.93,39,0,1.22,EG,1717694597
264,264,tura,25.5198,90.2201,296.93,91,0,1.71,IN,1717694662
523,523,yuzhno-kurilsk,44.0306,145.8556,278.91,93,0,1.56,RU,1717694815
329,329,luderitz,-26.6481,15.1594,287.87,82,0,1.65,NaN,1717694701
322,322,goondiwindi,-28.5500,150.3167,279.85,79,0,1.96,AU,1717694697
12,12,ushuaia,-54.8000,-68.3000,272.94,74,0,1.54,AR,1717694503
503,503,batroun,34.2553,35.6581,302.55,55,0,1.88,LB,1717694804
479,479,khasab,26.1799,56.2477,305.41,36,0,1.69,OM,1717694790
490,490,oravita,45.0325,21.6894,300.30,67,0,1.93,RO,1717694796


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =  my_cities_df[["City","Country","Lng","Lat","Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,column="Hotel Name",value="")

# Display sample data
hotel_df

,City,Country,Lng,Lat,Humidity,Hotel Name
12,ushuaia,AR,-68.3000,-54.8000,74,
120,dalhart,US,-102.5133,36.0595,29,
153,marsa alam,EG,34.8918,25.0757,39,
157,norman gardens,AU,150.5393,-23.3224,70,
206,sikandra,IN,79.6500,26.3667,15,
261,fortuna,US,-124.1573,40.5982,84,
264,tura,IN,90.2201,25.5198,91,
322,goondiwindi,AU,150.3167,-28.5500,79,
329,luderitz,NaN,15.1594,-26.6481,82,
351,motatei,RO,23.2000,44.0833,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}
# # Print a message to follow up the hotel search
print("Starting hotel search")

# # Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    longitude = row["Lng"]
    latitude = row["Lat"]
#   Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"
#   Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
#   Convert the API response to JSON format
    name_address = requests.get(base_url,params=params).json()
#   Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#   If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"      
#   Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
dalhart - nearest hotel: La Quinta Inn & Suites by Wyndham Dalhart
marsa alam - nearest hotel: Blue House Hotel
norman gardens - nearest hotel: Kalka Hotel
sikandra - nearest hotel: No hotel found
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
tura - nearest hotel: Hotel Polo Orchid
goondiwindi - nearest hotel: Queensland Hotel
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
motatei - nearest hotel: No hotel found
ad dabbah - nearest hotel: No hotel found
khasab - nearest hotel: "Extra Divers Villa"
oravita - nearest hotel: Condor
newman - nearest hotel: No hotel found
batroun - nearest hotel: Villa Paradiso
yuzhno-kurilsk - nearest hotel: Айсберг
santaquin - nearest hotel: Wee Blue Inn


,City,Country,Lng,Lat,Humidity,Hotel Name
12,ushuaia,AR,-68.3000,-54.8000,74,Apart Hotel Aires del Beagle
120,dalhart,US,-102.5133,36.0595,29,La Quinta Inn & Suites by Wyndham Dalhart
153,marsa alam,EG,34.8918,25.0757,39,Blue House Hotel
157,norman gardens,AU,150.5393,-23.3224,70,Kalka Hotel
206,sikandra,IN,79.6500,26.3667,15,No hotel found
261,fortuna,US,-124.1573,40.5982,84,Comfort Inn & Suites Redwood Country
264,tura,IN,90.2201,25.5198,91,Hotel Polo Orchid
322,goondiwindi,AU,150.3167,-28.5500,79,Queensland Hotel
329,luderitz,NaN,15.1594,-26.6481,82,Seaview Hotel Zum Sperrgebiet
351,motatei,RO,23.2000,44.0833,51,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    hover_cols = ["Hotel Name","Country"],
    geo = True,
    tiles = "OSM",
    color = "City",
    alpha = 0.90,
    frame_width = 700,
    frame_height = 500,
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)